### Imports

In [1397]:
from IPython.display import Image, display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19

In [1398]:
DIR = "drive/MyDrive/Colab Notebooks/Projects/Neural Style Transfer/"

### Preprocess the input

In [1399]:
content_image_path = keras.utils.get_file("paris.jpg", "https://i.imgur.com/F28w3Ac.jpg")

style_image_path = keras.utils.get_file("starry_night.jpg", "https://i.imgur.com/9ooB60I.jpg")

In [1400]:
# Images
content_image = keras.preprocessing.image.load_img(content_image_path)
style_image = keras.preprocessing.image.load_img(style_image_path)

In [1401]:
width, height = content_image.size

In [1402]:
# Target dimensions for generated image.
rows = 400
cols = int(width * (rows / height))

In [1403]:
print(rows, cols)

400 711


In [1404]:
# Convert image to tensor
def preprocess_image(image_path):
  img = keras.preprocessing.image.load_img(
      image_path, target_size=(rows, cols)
  )
  img = keras.preprocessing.image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = vgg19.preprocess_input(img)
  return tf.convert_to_tensor(img)

In [1405]:
# Convert tensor to image
def unprocess_image(X):
  X = X.reshape((rows, cols, 3))
  # Remove zero-center by mean pixel
  # VGG network requires images to be zero mean.
  X[:, :, 0] += 103.939
  X[:, :, 1] += 116.779
  X[:, :, 2] += 123.68
  # 'BGR' -> 'RGB'
  X = X[:, :, ::-1]
  X = np.clip(X, 0, 255).astype("uint8")
  return X

### Create the VGG19 model with pre-trained ImageNet weights, without the top layers.

In [1406]:
vgg = vgg19.VGG19(include_top=False, weights='imagenet')
vgg.trainable = False

In [1407]:
for layer in vgg.layers:
  print(layer.name)

input_142
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool


In [1408]:
# Content layers matched
content_layers = [
    'block4_conv1',
    #'block5_conv1'
    ]

# Style layers matched
style_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    #'block5_conv1'
    ]

In [1409]:
# Given a model, returns the feature outputs when called with 
# some image input.
class FeatureExtractor(keras.models.Model):
  def __init__(self, layer_names):
    super(FeatureExtractor, self).__init__()
    self.vgg = vgg19.VGG19(include_top=False, weights='imagenet')
    self.vgg.trainable = False
    self.outputs = dict([(name, self.vgg.get_layer(name).output) for name in layer_names])
    self.model = tf.keras.Model([self.vgg.input], self.outputs)

  def call(self, x):
    return self.model(x)

In [1410]:
content_extractor = FeatureExtractor(layer_names=content_layers)
style_extractor = FeatureExtractor(layer_names=style_layers)

In [1411]:
content_image_processed = preprocess_image(content_image_path)
style_image_processed = preprocess_image(style_image_path)

In [1412]:
content_outputs = content_extractor(content_image_processed)
style_outputs = style_extractor(style_image_processed)

In [1413]:
for layer_name in content_outputs:
  print(layer_name, content_outputs[layer_name].shape)

block4_conv1 (1, 50, 88, 512)


In [1414]:
for layer_name in style_outputs:
  print(layer_name, style_outputs[layer_name].shape)

block1_conv1 (1, 400, 711, 64)
block2_conv1 (1, 200, 355, 128)
block3_conv1 (1, 100, 177, 256)
block4_conv1 (1, 50, 88, 512)


### Helper functions

In [1415]:
# Returns the Gram matrix
def get_gram_matrix(X):
  C = X.get_shape().as_list()[-1]
  reshaped = tf.reshape(X, [-1, C])
  G = tf.matmul(reshaped, reshaped, transpose_a=True)
  return G

In [1416]:
# P = C(content), F = C(generated)
def calc_content_loss(P, F):
  #return tf.reduce_sum(tf.square(F - P)) / 2.0
  return tf.reduce_mean(tf.square(F - P)) / 2.0

# A = S(style), G = S(generated) for a layer
def calc_style_loss(A, G):
  A_gram = get_gram_matrix(A)
  G_gram = get_gram_matrix(G)
  shape = A.get_shape().as_list()
  N_l = shape[-1]
  M_l = shape[1] * shape[2]
  return tf.reduce_mean(tf.square(G_gram - A_gram)) / (4.0 * (N_l**2) * (M_l**2))

# From https://keras.io/examples/generative/neural_style_transfer/
# This is used for visual coherence
def total_variation_loss(X):
  A = tf.square(
      X[:, : rows - 1, : cols - 1, :] - X[:, 1:, : cols - 1, :]
  )
  B = tf.square(
      X[:, : rows - 1, : cols - 1, :] - X[:, : rows - 1, 1:, :]
  )
  return tf.reduce_sum(tf.pow(A + B, 1.25))
  #return tf.reduce_mean(tf.pow(A + B, 1.25))

In [1417]:
# The ratio alpha/beta is 1e-3 or 1e-4 in the paper

# Content weight
alpha = 1e1
# Style weight
beta = 1e7
# Total variation weight
total_variation_weight = 5e-2

Store the target features for the content image and style image. These only need to be calculated once since the network is not going to change.

In [1418]:
content_target = content_extractor(content_image_processed)
style_target = style_extractor(style_image_processed)

In [1419]:
N_content = len(content_layers)
N_style = len(style_layers)

In [1420]:
def compute_loss(x):
  content_features = content_extractor(x)
  style_features = style_extractor(x)

  # Content loss
  content_loss = 0.0
  for layer in content_features:
    content_loss += (calc_content_loss(content_target[layer], content_features[layer]) / N_content)

  # Style loss
  style_loss = 0.0
  for layer in style_features:
    style_loss += (calc_style_loss(style_target[layer], style_features[layer]) / N_style)
  
  loss = alpha*content_loss + beta*style_loss

  # Total variation loss
  loss += total_variation_weight * total_variation_loss(x)
  return loss

### Training

In [1421]:
lr = 1
EPOCHS = 500

In [1422]:
# Adam optimizer
opt_adam = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=1e-1)

In [1423]:
@tf.function
def train_step(x):
  with tf.GradientTape() as tape:
    loss = compute_loss(x)
  grads = tape.gradient(loss, x)
  return loss, grads

In [1424]:
combination_image = tf.Variable(content_image_processed)

def train(epochs):
  for epoch in range(1, epochs+1):
    loss, grads = train_step(combination_image)
    opt_adam.apply_gradients([(grads, combination_image)])

    if epoch % 100 == 0:
      clear_output()
      print(f'Iteration: {epoch}, loss: {loss}')
      
  img = unprocess_image(combination_image.numpy())
  fname = DIR + "/images/1-1.png"
  keras.preprocessing.image.save_img(fname, img)
  display(Image(fname))

In [ ]:
train(EPOCHS)